## The Mood of the News

In [1]:
#Dependencies
import pandas as pd
import tweepy
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [12]:
# Target Search Term
target_user = ("@BBCNews", "@CNNbrk", "@CBSNews","@FoxNews", "@nytimes")

# Array to hold sentiments
sentiments = []

# Convert tweet timestamps to datetime objects that can be manipulated by Python
converted_timestamps = []

# Variables for holding sentiments
compound_list = []

# Counter
counter = 1

# Loop through all target users
for target in target_user:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    neutral_list = []
    negative_list = []
    
    
    # Get all tweets from home feed
    public_tweets = api.user_timeline(target, count=100, result_type="recent", max_id=oldest_tweet)

    # Loop through all tweets
    for tweet in public_tweets:
            
        #Convert the raw tweet time to correct datetime stamp.
        converted_time = datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")

        # Run Vader Analysis on each tweet
        results = analyzer.polarity_scores(tweet["text"])
        compound = results["compound"]
        pos = results["pos"]
        neu = results["neu"]
        neg = results["neg"]
    
        # Add sentiments for each tweet into an array
        sentiments.append({"News Source": target,
                       "Date/Time": converted_time, 
                        "Compound": compound,
                        "Positive": pos,
                        "Neutral": neu,
                        "Negative": neg,
                        "Tweet": tweet["text"],
                        "Tweets Ago": counter})
        # Add to counter 
        counter = counter + 1

In [13]:
#Need to put tweet in df with time stamp, sentiment analysis and tweet. 
sentiments_df = pd.DataFrame.from_dict(sentiments)
sentiments_df.head()

,Compound,Date/Time,Negative,Neutral,News Source,Positive,Tweet,Tweets Ago
0,0.6808,2017-11-30 22:30:02+00:00,0.000,0.699,@BBCNews,0.301,"Friday's Times: ""Trump gave us boost in suppor...",1
1,-0.2500,2017-11-30 22:30:00+00:00,0.133,0.867,@BBCNews,0.000,"Friday's Guardian: ""PM joins storm of protest ...",2
2,-0.5719,2017-11-30 22:29:58+00:00,0.252,0.748,@BBCNews,0.000,"Friday's Daily Telegraph: ""Motorists 'misled' ...",3
3,0.0000,2017-11-30 22:29:54+00:00,0.000,1.000,@BBCNews,0.000,"Friday's Daily Mirror: ""Not wanted"" #tomorrows...",4
4,-0.4391,2017-11-30 22:28:34+00:00,0.224,0.776,@BBCNews,0.000,RT @BBCPolitics: Minister Sam Gyimah 'deeply u...,5


In [14]:
#Make the df look nice.  
news_sentiments = pd.DataFrame(sentiments_df, columns= ["News Source", "Date/Time", "Compound","Positive",
                                                        "Neutral", "Negative", "Tweet", "Tweets Ago"])
news_sentiments.head()

,News Source,Date/Time,Compound,Positive,Neutral,Negative,Tweet,Tweets Ago
0,@BBCNews,2017-11-30 22:30:02+00:00,0.6808,0.301,0.699,0.000,"Friday's Times: ""Trump gave us boost in suppor...",1
1,@BBCNews,2017-11-30 22:30:00+00:00,-0.2500,0.000,0.867,0.133,"Friday's Guardian: ""PM joins storm of protest ...",2
2,@BBCNews,2017-11-30 22:29:58+00:00,-0.5719,0.000,0.748,0.252,"Friday's Daily Telegraph: ""Motorists 'misled' ...",3
3,@BBCNews,2017-11-30 22:29:54+00:00,0.0000,0.000,1.000,0.000,"Friday's Daily Mirror: ""Not wanted"" #tomorrows...",4
4,@BBCNews,2017-11-30 22:28:34+00:00,-0.4391,0.000,0.776,0.224,RT @BBCPolitics: Minister Sam Gyimah 'deeply u...,5


In [15]:
#Export the dataframe to a csv file. 
news_sentiments.to_csv('New_Source_sentiments.csv', encoding='utf-8', index=False)

In [11]:
#Graph the 100 tweets from each news source with a timestamp "Tweets Ago". 

#news_source = news_sentiments["News Source"].unique()
#count = np.arange(0,len(news_source))

plt.plot(np.arange(len(news_sentiments["News Source"].unique())),
         news_sentiments["Compound"], marker="o", linewidth=0.5,
         alpha=0.8)

# # Incorporate the other graph properties

plt.title(f"Sentiment Analysis of Tweets {datetime.now().strftime('%Y-%m-%d %H:%M')}")
plt.ylabel("Tweet Polarity")
plt.xlabel("Tweets Ago")
plt.show()

ValueError: x and y must have same first dimension, but have shapes (5,) and (500,)